In [3]:
import numpy as np
from collections import defaultdict
def generate_input(name):
    l_string = open(name).read().rstrip().replace("\n\n", "\t").split("\t")
    l_string = [sub.split(":\n") for sub in l_string]
    return l_string

def generateDict(puzzleInput):
    dictionary={}
    for picture in puzzleInput:
        key=picture[0].split(" ")[1].replace(':','')
        pic=picture[1].split("\n")
        dictionary[key]=[list(sub) for sub in pic]        
    return dictionary

def getSeamonsterForm(name):
    return open(name).read().rstrip().splitlines()

def generateEdgesDict(puzzleDict):
    dictionary={}
    for picture in puzzleDict:
        dictionary=getEdges(puzzleDict[picture],dictionary,picture)     
    return dictionary

def getEdges(imageArray,dictionary,picture):
    up=imageArray[0]
    down=imageArray[-1]
    left,right= [],[]
    for subarray in imageArray:
        left.append(subarray[0])
        right.append(subarray[-1])
    dictionary[picture]={"up":up,"down":down,"left":left,"right":right}
    return dictionary

def day20p1(dictionary):
    summ=1
    corners={}
    bestmatch=defaultdict(list)
    for key in dictionary:
        count=0
        edges=[]
        for elements in dictionary[key]:
            for key2 in dictionary:
                if((not key==key2) 
                   and (dictionary[key][elements] in dictionary[key2].values() 
                        or dictionary[key][elements][::-1] in dictionary[key2].values())):
                        count+=1
                        bestmatch["".join(dictionary[key][elements])].append(key2)
                        bestmatch["".join(dictionary[key][elements])].append(key)
                        bestmatch["".join(dictionary[key][elements][::-1])].append(key2)
                        bestmatch["".join(dictionary[key][elements][::-1])].append(key)
                        edges.append(elements)
                        break
        if(count==2):
            summ*=int(key)
            corners|={key:edges}
    return summ,corners,bestmatch



def rotate90(dictionary,edge,d):
    d[edge]=np.rot90(d[edge]).tolist()
    dictionary=getEdges(d[edge],dictionary,edge)
    return dictionary,d

def vflip(dictionary,edge,d):
    d[edge]=np.flipud(d[edge]).tolist()
    dictionary=getEdges(d[edge],dictionary,edge)
    return dictionary,d

def hflip(dictionary,edge,d):
    d[edge]=np.fliplr(d[edge]).tolist()
    dictionary=getEdges(d[edge],dictionary,edge)
    return dictionary,d

def flipEdges(edge,dictionary,SearchDown,d,target):
    #print("edge="+str(edge))
    copydict,copyd=dictionary.copy(),d.copy()
    if(SearchDown):
        targetdirection="up"
    else:
        targetdirection="left"
    if(copydict[edge][targetdirection]==target):
        return True,copydict,d
    for _ in range(4):
        for _ in range(4):
            for _ in range(2):
                if(copydict[edge][targetdirection]==target):
                    return True,copydict,copyd
                copydict,copyd=hflip(copydict,edge,d)
            copydict,copyd=vflip(copydict,edge,copyd)
        copydict,copyd=rotate90(copydict,edge,copyd)   
    return False,copydict,copyd


def createGrid(dictionary,corners,imageLenght,d,bestmatch):
    grid=[[None]*imageLenght for _ in range(imageLenght)]
    topleft=''
    for key, corn in corners.items():
        if(corn==['down', 'right']):
            topleft=key
            grid[0][0]=topleft
            break    
    grid[0][0]=topleft
    for i in range(imageLenght):
        #print("i="+str(i))
        for j in range(imageLenght-1):
            #print("j="+str(j))
            targetRight=dictionary[grid[i][j]]["right"]
            rightKey=bestmatch["".join(targetRight)].copy()
            rightKey.remove(grid[i][j])
            rightKey=rightKey[0]
            isFind,dictionary,d=flipEdges(rightKey,dictionary,False,d,targetRight)
            grid[i][j+1]=rightKey
            if(not isFind):
                print("FUCK YOURSELF ERIC")
                return None,None   
        if(not i==(len(grid)-1)): 
            targetDown=dictionary[grid[i][0]]["down"]
            downKey=bestmatch["".join(targetDown)].copy()
            downKey.remove(grid[i][0])
            downKey=downKey[0]
            isFind,dictionary,d=flipEdges(downKey,dictionary,True,d,targetDown)
            grid[i+1][0]=downKey
            if(not isFind):
                print("FUCK YOURSELF ERIC")
                return None,None    
    return np.array(grid)

def countseamonster(formedImage,indexes):
    laCreaturaCount=0
    lengthcol=len(formedImage)-3
    matchlength=len(indexes)
    lengthrow=len(formedImage[0])-20
    for i in range(lengthcol):
        for j in range(lengthrow):
            match=0
            for i2,j2 in indexes:
                if(formedImage[i+i2][j+j2]=="#"):
                    match+=1
                else:
                    break
            if(match==matchlength):
                laCreaturaCount+=1
                for i2,j2 in indexes:
                    formedImage[i+i2][j+j2]="O"
    return laCreaturaCount
            

def search_seamonster(formedImage,indexes):
    for _ in range(4):
        for _ in range(4):
            for _ in range(2):
                seacount=countseamonster(formedImage,indexes)
                if(seacount>0):
                    print("There are "+str(seacount)+" seamonsters")
                    print(str(somme-(seacount)*len(indexes))+" '#' are not part of a sea monster \n")
                    return formedImage
                formedImage=np.fliplr(formedImage)
            formedImage=np.flipud(formedImage)
        formedImage=np.rot90(formedImage)  
        

puzzleInput=generate_input("input.txt")
imageLenght=int(len(puzzleInput)**(1/2))
d=generateDict(puzzleInput)
edges=generateEdgesDict(d)
##Part 1
summ,corners,bestmatch=day20p1(edges)
print("The four corner tiles are "+str(list(corners.keys())))
print("The product of those corners is: "+str(summ))
bestmatch = {k:list(dict.fromkeys(v)) for k, v in bestmatch.items() if len(v) >= 2}

#sum([elem.count("#") for elem in it for key,it in d.items()])
grid=createGrid(edges,corners,int(imageLenght),d,bestmatch)
#print(grid)
image=[[None]*imageLenght for _ in range(imageLenght)]
for i,imagerow in enumerate(grid):
    #print(imagerow)
    for j,imageid in enumerate(imagerow):
        #print(imageid)
        #print(d[grid[i][j]])
        #print(i,j)
        image[i][j]= np.array([''.join(a) for a in d[grid[i][j]] ]).reshape(10,1).tolist()
        #image[i][j]='\n'.join(image[i][j])



for i,line in enumerate(image): 
    for j,row in enumerate(line):
        newrow=[]
        for h in range(10):
            newrow.append([row[h][0][1:-1]])
        image[i][j]=newrow


formedImage=[]
for line in image:
    for i in range(1,9):
        l=''
        for row in line:
            l+=row[i][0]
        formedImage.append(l)

somme=0
for elm in formedImage:
    somme+=elm.count('#')
#2530
for i,elm in enumerate(formedImage):
    formedImage[i]=list(elm)

seamonsterForm=getSeamonsterForm("seamonster.txt")
indexes=[]
for i,x in enumerate(seamonsterForm):
    for j,y in enumerate(x):
        if(y=='#'):
            indexes.append((i,j))

###Get seamonster
formedImage=search_seamonster(formedImage,indexes)
for row in formedImage:
    print("".join(row))


The four corner tiles are ['3433', '3833', '2011', '3001']
The product of those corners is: 79412832860579
There are 25 seamonsters
2155 '#' are not part of a sea monster 

........................##......#..##...#...#........##.......#..#.......#..#...##.....###....#.
##.....##.....#..##...#...#.....O....#.##...#...#...#..##....#...#.#..#...#...O##.............#.
.......#.#....O...#OO...#OO##..OOO#..##........##..#........O##..OO#...OO...#OOO...##.......#...
.###...##.#..#.O..O##O..O..O#.O..........###...##.#..##......O..O..O.#O..O.#O.##...........###..
....#..#.##..........#.#..#.#.......##...#..###..#.#.##..#....#.........#..##..#..#..#..#.....#.
#....##.......................#...#.........##........#........#.#.........#.#..###....#..#.#..#
.......#...##...#......###..........#.##.#.#.#...#....##...#.#..#......###.........##....#..#.#.
###......#..####.......#...#...#......##........#.....#...#....##..#...#........#.....#...#..#.#
#.....#..###........##..#.........#....#...#.......

In [104]:
up,down,left,right="up","down","left","right"
{"up":down[::-1],"down": up[::-1],"left":left[::-1],"right":right[::-1]}
{"up":down[::-1],"down": up[::-1],"left":left[::-1],"right":right[::-1]}
#v=d["3833"].copy()
len(seamonsterForm[0])
#print(np.array(v))
#print('\n\n\n')
#print(np.array(v))
#print('\n')
#print(np.array(v[::-1]))
#print('\n\n\n')
#print(np.array(v))
#print('\n')
#print(np.array([x[::-1] for x in v]))
#print('##..#...##'[::-1])

20

In [ ]:
import numpy as np
from collections import defaultdict
def generate_input(name):
    l_string = open(name).read().rstrip().replace("\n\n", "\t").split("\t")
    l_string = [sub.split(":\n") for sub in l_string]
    return l_string

def generateDict(puzzleInput):
    dictionary={}
    for picture in puzzleInput:
        #arr=picture[1:].split("\n")
        #arr=np.array([list(sub) for sub in test2])
        key=picture[0].split(" ")[1].replace(':','')
        #value=np.array([[]])
        #for i in range(0,4):
        #    rotated=np.rot90(arr, k=i)
        #    np.append(value,[rotated])
        pic=picture[1].split("\n")
        dictionary[key]=[list(sub) for sub in pic]
        
    return dictionary


def generateEdgesDict(puzzleDict):
    dictionary={}
    for picture in puzzleDict:
        dictionary=getEdges(puzzleDict[picture],dictionary,picture)
            
    return dictionary
def getEdges(imageArray,dictionary,picture):
    up=imageArray[0]
    down=imageArray[-1]
    left,right= [],[]
    for subarray in imageArray:
        left.append(subarray[0])
        right.append(subarray[-1])
    dictionary[picture]={"up":up,"down":down,"left":left,"right":right}
    return dictionary

def day20p1(dictionary):
    summ=1
    corners={}
    bestmatch=defaultdict(list)
    for key in dictionary:
        count=0
        edges=[]
        for elements in dictionary[key]:
            for key2 in dictionary:
                if((not key==key2)):
                    if(dictionary[key][elements] in dictionary[key2].values()):
                        count+=1
                        bestmatch["".join(dictionary[key][elements])].append(key2)
                        bestmatch["".join(dictionary[key][elements])].append(key)
                        bestmatch["".join(dictionary[key][elements][::-1])].append(key2)
                        bestmatch["".join(dictionary[key][elements][::-1])].append(key)
                        edges.append(elements)
                        break
                    elif(dictionary[key][elements][::-1] in dictionary[key2].values()):
                        count+=1
                        bestmatch["".join(dictionary[key][elements][::-1])].append(key2)
                        bestmatch["".join(dictionary[key][elements][::-1])].append(key)
                        bestmatch["".join(dictionary[key][elements])].append(key2)
                        bestmatch["".join(dictionary[key][elements])].append(key)
                        edges.append(elements)
                        break
        if(count==2):
            summ*=int(key)
            corners|={key:edges}
    return summ,corners,bestmatch

def flipEdges(edge,dictionary,nbRotation,isReverse,placed,Isvertical):
    up,down= dictionary[edge]["up"],dictionary[edge]["down"]
    left,right=dictionary[edge]["left"],dictionary[edge]["right"]
    if(isReverse):
        if(not Isvertical):
            up,down,left,right=down,up,left[::-1],right[::-1]
        else:
            up,down,left,right=up[::-1],down[::-1],right,left
    dictionary[edge]={"up":up,"down":down,"left":left,"right":right}
    getPossiblematch(dictionary,edge,placed)
    print("flip")
    for i in range(nbRotation):
        up,down,left,right=left[::-1],right[::-1],down,up
    dictionary[edge]={"up":up,"down":down,"left":left,"right":right}
    e=getPossiblematch(dictionary,edge,placed)
    return dictionary

def isGood(key,direction,r,isRotate,dictionary,corners):
    rotate={0:"left",3:"left",2:"right","right":1}
    if(getNeighbors(dictionary,dictionary[key][rotate[r]],rotate[r],[key],key,corners) is None):
        return True
    else:
        return False
        

def flipEdges2(edge,dictionary,nbRotation,isReverse,placed,Isvertical):
    up,down= dictionary[edge]["up"],dictionary[edge]["down"]
    left,right=dictionary[edge]["left"],dictionary[edge]["right"]
    if(isReverse):
        up,down,left,right=up[::-1],down[::-1],left[::-1],right[::-1]
        getPossiblematch(dictionary,edge,placed)
        print("flip")
    dictionary[edge]={"up":up,"down":down,"left":left,"right":right}
    for i in range(nbRotation):
        up,down,left,right=left,right,down,up
    dictionary[edge]={"up":up,"down":down,"left":left,"right":right}
    e=getPossiblematch(dictionary,edge,placed)
    print("rotated")
    if(isReverse):
        if(not Isvertical and ("up" in e)):
            dictionary[edge]={"up":down,"down":up,"left":left,"right":right}
            getPossiblematch(dictionary,edge,placed)
            print("vertflip")
        elif(Isvertical and ("left" in e)):
            dictionary[edge]={"up":up,"down":down,"left":right,"right":left}
            getPossiblematch(dictionary,edge,placed)
            print("horiflip")
    
    return dictionary


def rotate90(dictionary,edge,d):
    #up,down= dictionary[edge]["up"],dictionary[edge]["down"]
    #left,right=dictionary[edge]["left"],dictionary[edge]["right"]
    #dictionary[edge]={"up":left,"down": right,"left":down,"right":up}
    #d[edge]=list(zip(*d[edge]))[::-1]
    d[edge]=np.rot90(d[edge]).tolist()
    dictionary=getEdges(d[edge],dictionary,edge)
    return dictionary,d

def vflip(dictionary,edge,d):
    #up,down= dictionary[edge]["up"],dictionary[edge]["down"]
    #left,right=dictionary[edge]["left"],dictionary[edge]["right"]   
    #dictionary[edge]={"up":down,"down": up,"left":left[::-1],"right":right[::-1]}
    #d[edge]=d[edge][::-1]
    d[edge]=np.flipud(d[edge]).tolist()
    dictionary=getEdges(d[edge],dictionary,edge)
    return dictionary,d

def hflip(dictionary,edge,d):
    #up,down= dictionary[edge]["up"],dictionary[edge]["down"]
    #left,right=dictionary[edge]["left"],dictionary[edge]["right"]   
    #dictionary[edge]={"up":up[::-1],"down": down[::-1],"left":right,"right":left }
    #d[edge]=[x[::-1] for x in d[edge]] 
    d[edge]=np.fliplr(d[edge]).tolist()
    dictionary=getEdges(d[edge],dictionary,edge)
    return dictionary,d

def flipEdges3(edge,dictionary,nbRotation,isReverse,placed,SearchDown,target,d):
    copydict,copyd=dictionary.copy(),d.copy()
    if(SearchDown):
        targetdirection="up"
    else:
        targetdirection="left"
    if(copydict[edge][targetdirection]==target):
        return True,copydict,d
    
    for _ in range(4):
        copydict,copyd=rotate90(copydict,edge,copyd)
        if(copydict[edge][targetdirection]==target):
            return True,copydict,copyd
    
    copydict,d=hflip(copydict,edge,d)
    for _ in range(4):
        copydict,copyd=rotate90(copydict,edge,copyd)
        if(copydict[edge][targetdirection]==target):
            return True,copydict,copyd    
    
    copydict,d=vflip(copydict,edge,copyd)
    for _ in range(4):
        copydict,copyd=rotate90(copydict,edge,copyd)
        if(copydict[edge][targetdirection]==target):
            return True,copydict,copyd         
    
    copydict,d=hflip(copydict,edge,copyd)
    for _ in range(4):
        copydict,copyd=rotate90(copydict,edge,copyd)
        if(copydict[edge][targetdirection]==target):
            return True,copydict,copyd  
        
    return False,dictionary,d

def flipEdges4(edge,dictionary,nbRotation,isReverse,placed,SearchDown,d,targetID):
    copydict,copyd=dictionary.copy(),d.copy()
    if(SearchDown):
        target=copydict[targetID]["down"]
        print(target)
        targetdirection="up"
    else:
        target=copydict[targetID]["right"]
        print(target)
        targetdirection="left"
    if(copydict[edge][targetdirection]==target):
        return True,copydict,d
    
    for _ in range(4):
        for _ in range(4):
            for _ in range(2):
                if(copydict[edge][targetdirection]==target):
                    return True,copydict,copyd
                copydict,d=hflip(copydict,edge,d)
            copydict,d=vflip(copydict,edge,copyd)
        copydict,copyd=rotate90(copydict,edge,copyd)   
    return False,dictionary,d

def getPossiblematch(dictionary,currentKey,placed,bestmatch):
    edges=[]
    for elements in dictionary[currentKey]:
        #print(elements)
        for key2 in dictionary:
            if(key2 in placed):
                continue
            if( (not currentKey==key2) and ( dictionary[currentKey][elements] in dictionary[key2].values() or dictionary[currentKey][elements][::-1] in dictionary[key2].values() ) ):
                print(elements)
                print("".join(dictionary[currentKey][elements]))
                print("\n")
                if( ("".join(dictionary[currentKey][elements]) not in bestmatch 
                   or "".join(dictionary[currentKey][elements][::-1]) not in bestmatch ) ):
                    bestmatch["".join(dictionary[key][elements])].append(key2)
                    bestmatch["".join(dictionary[key][elements])].append(currentKey)
                    bestmatch["".join(dictionary[key][elements][::-1])].append(key2)
                    bestmatch["".join(dictionary[key][elements][::-1])].append(currentKey)
                    bestmatch = {k:list(dict.fromkeys(v)) for k, v in bestmatch.items() if len(v) >= 2}
                edges.append(elements)
                break
    print("edges="+str(edges))
    return edges,bestmatch

def getNeighbors(dictionary,target,direction,placed,currentKey,corners):
    if(direction=="right"):
        rotate={"left":0,"up":3,"right":2,"down":1}
    else:
        #rotate={"left":(3,1),"up":(0,0),"right":(1,3),"down":(2,2)}
        rotate={"left":1,"up":0,"right":3,"down":2}
    getPossiblematch(dictionary,currentKey,placed)
    for i,key in enumerate(dictionary):
        if(key in placed):
            continue
        for j,dirr in enumerate(dictionary[key]):
            if(dictionary[key][dirr]==target):
                #if(isGood(key,direction,rotate[direction],False,dictionary,corners) and not key in corners):
                #    continue
                print(dirr,rotate[dirr])
                return key,rotate[dirr],False
            elif(dictionary[key][dirr]==target[::-1]):
                #if(isGood(key,direction,rotate[direction],True,dictionary,corners) and not key in corners):
                #    continue
                print('reverted')
                print(dirr,rotate[dirr])
                return key,rotate[dirr],True


    
def createGrid(dictionary,corners,imageLenght,d,bestmatch):
    grid=[[None]*imageLenght for _ in range(imageLenght)]
    #rotateReverse=[[None]*imageLenght for _ in range(imageLenght)]
    topleft=''
    placed=[]
    print(corners)
    for key, corn in corners.items():
        if(corn==['down', 'right']):
            topleft=key
            grid[0][0]=topleft
            break
    
    #grid[0][0]='1117'
    grid[0][0]=topleft
    #flipEdges(grid[0][0],dictionary,2,False)
    #rotateReverse[0][0]=(0,False)
    placed.append(grid[0][0])
    for i in range(imageLenght):
        print("i="+str(i))
        print("current key=")
        getPossiblematch(dictionary,grid[i][0],placed,bestmatch)
        #dictionary=getEdges(d[grid[i][0]],dictionary,grid[i][0])
        for j in range(imageLenght-1):
            print("j="+str(j))
            #dictionary=getEdges(d[grid[i][j]],dictionary,grid[i][j])
            #print(dictionary[grid[i][j]])
            print(dictionary[grid[i][j]]['right'])
            targeRight=dictionary[grid[i][j]]["right"]
            print("".join(targeRight))
            #targeDown=dictionary[title]["down"]
            rightKey,r,isReverse=bestmatch["".join(targeRight)].copy(),None,None
            print(rightKey)
            rightKey.remove(grid[i][j])
            print(rightKey)
            rightKey=rightKey[0]
            #getNeighbors(dictionary,targeRight,"right",placed,grid[i][j],corners)
            print("next key=")
            bidon,bestmatch=getPossiblematch(dictionary,rightKey,placed,bestmatch)
            isFind,dictionary,d=flipEdges4(rightKey,dictionary,r,isReverse,placed,False,d,grid[i][j])
            print("next key=")
            bidon,bestmatch=getPossiblematch(dictionary,rightKey,placed,bestmatch)
            grid[i][j+1]=rightKey
            #rotateReverse[i][j+1]=(r,isReverse)
            placed.append(rightKey)
            print(rightKey)
            print(np.array(grid))
            if(not isFind):
                print("FUCK YOURSELF ERIC")
                return None,None
            print('\n')
        
        if(not i==(len(grid)-1)): 
            #dictionary=getEdges(d[grid[i][0]],dictionary,grid[i][0])
            #for each in dictionary[grid[i][0]]:
            #    print(each)
            #    print(bestmatch["".join(dictionary[grid[i][0]][each])])
            targeDown=dictionary[grid[i][0]]["down"]
            print("down")
            print(targeDown)
            print("".join(targeDown))
            print(grid[i][0])
            print("next key=")
            bidon,bestmatch=getPossiblematch(dictionary,grid[i][0],placed,bestmatch)
            #downKey,r,isReverse=getNeighbors(dictionary,targeDown,"down",placed,grid[i][0],corners)
            downKey,r,isReverse=bestmatch["".join(targeDown)].copy(),None,None
            print(downKey)
            downKey.remove(grid[i][0])
            print(downKey)
            downKey=downKey[0]
            print("next key=")
            bidon,bestmatch=getPossiblematch(dictionary,downKey,placed,bestmatch)
            isFind,dictionary,d=flipEdges4(downKey,dictionary,r,isReverse,placed,True,d,grid[i][0])
            print("next key=")
            bidon,bestmatch=getPossiblematch(dictionary,downKey,placed,bestmatch)
            grid[i+1][0]=downKey
            #rotateReverse[i+1][0]=(r,isReverse)
            placed.append(downKey)
            print("downkey="+str(downKey))
            print(isReverse)
            print(np.array(grid))
            if(not isFind):
                print("FUCK YOURSELF ERIC")
                return None,None
            print("\n\n\n")
    
    return np.array(grid)
    
puzzleInput=generate_input("input.txt")
#print(puzzleInput[1:])
imageLenght=int(len(puzzleInput)**(1/2))
imageLenght
d=generateDict(puzzleInput)
#print(d)
"1579"
"2311"
#test=d["1579"]
#test2=test
#np.array(test2).shape
#test3=np.array([list(sub) for sub in test2])
#print(test3)
print('\n')
#print(np.rot90(test3))
#print('\n')
#print(np.flip(test3))
#print('\n\n\n')
edges=generateEdgesDict(d)
summ,corners,bestmatch=day20p1(edges)
print(summ)
bestmatch = {k:list(dict.fromkeys(v)) for k, v in bestmatch.items() if len(v) >= 2}
#print(bestmatch)
print("\n")
#print(edges['2749'])
#for each in edges['2749']:
#    print(each)
#    print(bestmatch["".join(edges['2749'][each])])
#arr=[list(sub) for key in d]
#print(arr[0])
#print(d['1579'])
somme=0
#for key in d:
#    for elem in d[key]:
#        somme+=elem.count('#')
        
#somme
#sum([elem.count("#") for elem in it for key,it in d.items()])
grid=createGrid(edges,corners,int(imageLenght),d,bestmatch)
print(grid)
#image=[[None]*imageLenght for _ in range(imageLenght)]
#im=[]
#for g in grid:
#np.array(image).shape